In [4]:
#!pip install dash==1.19.0  

#!pip install jupyter_dash 

 #!pip install --upgrade plotly

In [3]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px

In [5]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv('spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
launch_list=list(spacex_df['Launch Site'].unique())
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
    
                                 dcc.Dropdown(id='site-dropdown', 
                   options=[{'label': 'All Sites', 'value': 'ALL'}]+[
                           {'label': i, 'value': i} for i in launch_list
                           ],
                  value='ALL',
                  placeholder='Select a launch site',
                  searchable=True,
                  style={'width': '80%', 'padding':'3px', 'font-size':'20px', 'text-align-last':'center'}),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                         min=0, max=10000, step=1000,
                                         marks={0: '0', 2500: '2500',5000:'5000',7500:'7500',10000:'10000'},
                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site']==str(entered_site)]['class'].value_counts(normalize=True)
    if entered_site == 'ALL':

        fig = px.pie(values=spacex_df.groupby('Launch Site')['class'].mean(), 
        names=spacex_df.groupby('Launch Site')['Launch Site'].first(),
        title='Total Success Launches By Site')
        return fig
    else: 
        fig = px.pie(values=filtered_df, 
        names=spacex_df['class'].unique(), 
        title='Total Success Launches for site {}'.format(entered_site))
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")] )
def get_scatter_chart(entered_site,entered_payload):
    filt_df=spacex_df[(spacex_df['Launch Site']==str(entered_site)) &
                             (spacex_df['Payload Mass (kg)']>=entered_payload[0]) &
                               (spacex_df['Payload Mass (kg)']<=entered_payload[1]) ]
    filt_df_1=spacex_df[(spacex_df['Payload Mass (kg)']>=entered_payload[0]) &
                               (spacex_df['Payload Mass (kg)']<=entered_payload[1]) ]

    if entered_site == 'ALL':   
        fig = px.scatter(filt_df_1,
        x="Payload Mass (kg)", y="class", color="Booster Version Category",
        title='Correlation Between Payload and Success for All Sites')
        return fig
    else:
        fig = px.scatter(filt_df, 
        x="Payload Mass (kg)", y="class", color="Booster Version Category",hover_data=['Launch Site'],
        title='Correlation Between Payload and Success for {}'.format(entered_site)) 
        return fig                                 
                                   
    
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

/Applications/anaconda3/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

